# 1. Main Process:
- The Main Process is the primary process that is created when a Python program starts execution. It serves as the entry point for the program and is responsible for orchestrating the entire program's workflow, including managing resources, spawning child processes, and ensuring orderly termination.

## 1.1 Lifecycle of the Main Process:
#### i. Creation:
- The Main Process is automatically created when the Python interpreter starts.
- Its memory space and resources are allocated, and the entry point (if __name__ == "__main__":) is executed.
#### ii. Execution:
- The Main Process executes all the instructions in the Python script sequentially.
- During execution, it can create threads or processes for concurrent or parallel tasks.
#### iii. Waiting for Child Processes:
- The Main Process waits for all non-daemon child processes to complete using the join() method implicitly or explicitly.
- Daemon processes are terminated abruptly when the Main Process exits.
#### iv. Termination:
- Once all tasks are completed, the Main Process cleans up resources, including:
- Closing open file descriptors.
- Releasing locks or memory.

### 1.2 Responsibilities of the Main Process:
#### i. Spawning and Managing Child Processes:
- The Main Process creates child processes using multiprocessing.Process or higher-level abstractions like Pool or concurrent.futures.
#### ii. Resource Management:
- It handles memory allocation and deallocation, ensuring that the Python interpreter works efficiently.
#### iii. Synchronization:
- The Main Process can use synchronization primitives like Lock, Semaphore, and Condition to coordinate between child processes.
#### iv. Error Handling:
- If an exception occurs in the Main Process, it may propagate to child processes, causing them to terminate or enter an inconsistent state.

In [ ]:
import multiprocessing
import os
import time

def worker_task(name):
    print(f"Worker {name} started. PID: {os.getpid()}")
    time.sleep(2)
    print(f"Worker {name} finished.")

if __name__ == "__main__":
    print(f"Main Process started. PID: {os.getpid()}")

    # Create child processes
    processes = []
    for i in range(3):
        process = multiprocessing.Process(target=worker_task, args=(f"Process-{i+1}",))
        processes.append(process)
        process.start()

    # Wait for all child processes to complete
    for process in processes:
        process.join()

    print("Main Process finished.")


Main Process started. PID: 2087
Worker Process-1 started. PID: 2274
Worker Process-2 started. PID: 2277
Worker Process-3 started. PID: 2282
Worker Process-1 finished.
Worker Process-2 finished.
Worker Process-3 finished.
Main Process finished.


# 2. Daemon Process:

A Daemon Process is a background process designed to perform auxiliary tasks, such as logging, monitoring, or periodic cleanup. In Python, daemon processes are subordinate to the Main Process and have the following unique behavior:

- A daemon process will terminate abruptly when the Main Process exits, regardless of its state (finished or not).
- Daemon processes are often used for tasks that do not require orderly shutdown or completion.

### 2.1 Lifecycle of a Daemon Process:
#### i. Creation:
- Created like a regular process using multiprocessing.Process
- Its daemon attribute is explicitly set to True before calling start().
#### ii. Execution:
- The daemon process executes its target function or code.
- The Main Process does not wait for the daemon process to complete.
#### iii. Termination:
- When the Main Process exits:
- The daemon process is abruptly terminated without cleanup.
- Any incomplete tasks in the daemon process are abandoned.

## 1. Scenario: Teaching Sessions and Exam using multiprocessing
### 1.1 Using Non-Daemon Process

In [5]:
from multiprocessing import Process
import time
import os

def display():
    for i in range(10):
        print(f"Teaching session (PID: {os.getpid()}):", i)
        time.sleep(0.7)

if __name__ == "__main__":
    # Create a non-daemon process
    p1 = Process(target=display)
    p1.start()

    # Main process handling the Exam
    print("Exam Time!")
    time.sleep(3)  # Simulate exam process
    print("Exam is over")


Teaching session (PID: 6793): 0
Exam Time!
Teaching session (PID: 6793): 1
Teaching session (PID: 6793): 2
Teaching session (PID: 6793): 3
Teaching session (PID: 6793): 4
Exam is over


### Issues with Non-Daemon Process:
- The main process (Exam) finishes its work, but the program doesn’t exit because the non-daemon child process (Teaching Session) continues running until it completes all iterations.
- This causes unnecessary delays after the main process has completed its task.


### 1.2 Using Daemon Process

In [ ]:
from multiprocessing import Process
import time
import os

def display():
    for i in range(10):
        print(f"Teaching session (PID: {os.getpid()}):", i)
        time.sleep(0.7)

if __name__ == "__main__":
    # Create a daemon process
    p1 = Process(target=display, daemon=True)
    p1.start()

    # Main process handling the Exam
    print("Exam Time!")
    time.sleep(3)  # Simulate exam process
    print("Exam is over")


Teaching session (PID: 3035): 0
Exam Time!
Teaching session (PID: 3035): 1
Teaching session (PID: 3035): 2
Teaching session (PID: 3035): 3
Teaching session (PID: 3035): 4
Exam is over


### Advantages of Daemon Process:
- Non-Blocking: The main process does not block and can finish its work without waiting for the daemon process to complete.
- Automatic Termination: When the main process exits, all daemon processes are terminated automatically, even if they haven't completed their tasks.
- Efficient Background Tasks: Ideal for lightweight background tasks that do not need to finish before the program exits (e.g., monitoring, housekeeping).


### Key Points:
- Daemon State: A process can only be set as a daemon before it is started. Once a process starts, its daemon state cannot be changed.
- Main Process: The main process cannot be set as a daemon because it is required for program execution.

### Use Cases for Daemon Processes:
- Background tasks like logging, monitoring, or data cleanup, where the program's main functionality should not depend on their completion.